In [ ]:
%sql
CREATE SCHEMA IF NOT EXISTS bronze;
CREATE VOLUME IF NOT EXISTS bronze.ipl_archive;


In [ ]:
%sql
CREATE TABLE IF NOT EXISTS bronze.match_loading (
  match_id INT,
  season INT,
  city STRING,
  date STRING,
  team1 STRING,
  team2 STRING,
  toss_winner STRING,
  toss_decision STRING,
  result STRING,
  dl_applied INT,
  winner STRING,
  win_by_runs INT,
  win_by_wickets INT,
  player_of_match STRING,
  venue STRING,
  umpire1 STRING,
  umpire2 STRING,
  umpire3 STRING,
  home_team STRING,
  source_file STRING,
  ingestion_ts TIMESTAMP
)
USING DELTA;


In [ ]:
%sql
CREATE TABLE IF NOT EXISTS bronze.delivery_loading (
  match_id INT,
  inning INT,
  batting_team STRING,
  bowling_team STRING,
  over INT,
  ball INT,
  batsman STRING,
  non_striker STRING,
  bowler STRING,
  is_super_over INT,
  wide_runs INT,
  bye_runs INT,
  legbye_runs INT,
  noball_runs INT,
  penalty_runs INT,
  batsman_runs INT,
  extra_runs INT,
  total_runs INT,
  player_dismissed STRING,
  dismissal_kind STRING,
  fielder STRING,
  source_file STRING,
  ingestion_ts TIMESTAMP
)
USING DELTA;


In [ ]:
%sql
CREATE TABLE IF NOT EXISTS silver.dim_venue (
  venue_id BIGINT
    GENERATED ALWAYS AS IDENTITY,   -- Auto-increment surrogate key

  official_name STRING NOT NULL,
  city STRING NOT NULL,
  country STRING,

  capacity INT,
  latitude DECIMAL(10,8),
  longitude DECIMAL(11,8),

  created_at TIMESTAMP
)
USING DELTA;


In [ ]:
%sql
CREATE TABLE IF NOT EXISTS silver.dim_umpire (
  umpire_sk BIGINT
    GENERATED ALWAYS AS IDENTITY,   -- Surrogate key

  umpire_name STRING NOT NULL,      -- Natural key
  date_of_birth DATE,               -- DOB (core identity)
  country STRING,                   -- Nationality

  created_at TIMESTAMP
)
USING DELTA;


In [ ]:
%sql
CREATE TABLE IF NOT EXISTS silver.dim_team (
  team_sk BIGINT
    GENERATED ALWAYS AS IDENTITY,   -- Surrogate key
  team_name STRING NOT NULL,        -- Natural key
  short_name STRING,                -- CSK, RCB, MI
  active_flag BOOLEAN,
  created_at TIMESTAMP
)
USING DELTA;

In [ ]:
%sql
CREATE TABLE IF NOT EXISTS silver.dim_player (
  player_sk BIGINT
    GENERATED ALWAYS AS IDENTITY,   -- Surrogate key

  player_name STRING NOT NULL,      -- Natural key
  date_of_birth DATE,
  country STRING,
  role STRING,                      -- Batsman / Bowler / All-Rounder
  is_wicketkeeper BOOLEAN,          -- True if player keeps wickets
  batting_hand STRING,              -- Right / Left
  bowling_style STRING,             -- e.g., Right-arm fast
  retirement_year INT,              -- Null if active
  ipl_debut_year INT,               -- Null if not yet played
  created_at TIMESTAMP
)
USING DELTA;


In [ ]:
%sql
CREATE TABLE IF NOT EXISTS silver.fact_deliveries (
  delivery_sk BIGINT
    GENERATED ALWAYS AS IDENTITY,

  match_id INT,
  inning INT,
  over INT,
  ball INT,
  is_super_over INT,

  batting_team_sk BIGINT,
  bowling_team_sk BIGINT,

  batsman_sk BIGINT,
  non_striker_sk BIGINT,
  bowler_sk BIGINT,
  player_dismissed_sk BIGINT,
  fielder_sk BIGINT,

  wide_runs INT,
  bye_runs INT,
  legbye_runs INT,
  noball_runs INT,
  penalty_runs INT,
  batsman_runs INT,
  extra_runs INT,
  total_runs INT,

  dismissal_kind STRING,

  source_file STRING,
  ingestion_ts TIMESTAMP,
  created_at TIMESTAMP
)
USING DELTA
TBLPROPERTIES (
  delta.autoOptimize.optimizeWrite = true,
  delta.autoOptimize.autoCompact = true
);


In [ ]:
%sql
CREATE TABLE IF NOT EXISTS silver.fact_matches (
  match_sk BIGINT
    GENERATED ALWAYS AS IDENTITY,

  -- Degenerate dimension
  match_id INT,

  season INT,
  match_date DATE,
  dl_applied INT,

  -- Team dimensions
  team1_sk BIGINT,
  team2_sk BIGINT,
  toss_winner_sk BIGINT,
  winner_sk BIGINT,
  home_team_sk BIGINT,

  toss_decision STRING,
  result STRING,

  win_by_runs INT,
  win_by_wickets INT,

  -- Player dimension
  player_of_match_sk BIGINT,

  -- Venue & umpires
  venue_sk BIGINT,
  umpire1_sk BIGINT,
  umpire2_sk BIGINT,
  umpire3_sk BIGINT,

  -- Audit
  source_file STRING,
  ingestion_ts TIMESTAMP,
  created_at TIMESTAMP
)
USING DELTA
TBLPROPERTIES (
  delta.autoOptimize.optimizeWrite = true,
  delta.autoOptimize.autoCompact = true
);


In [ ]:
dbutils.fs.rm(
    "abfss://ipl-incremental-data@incrementalipldata.dfs.core.windows.net/row/match/",
    recurse=True
)
dbutils.fs.rm(
    "abfss://ipl-incremental-data@incrementalipldata.dfs.core.windows.net/row/delivery/",
    recurse=True
)

dbutils.fs.rm("/mnt/delta/delivery/checkpoints", recurse=True)
dbutils.fs.rm("/mnt/checkpoints/match", recurse=True)

True

In [ ]:
%sql
truncate table bronze.match_loading;
truncate table bronze.delivery_loading;
truncate table silver.fact_matches;
truncate table silver.fact_deliveries;
truncate table silver.dim_team;
truncate table silver.dim_umpire;
truncate table silver.dim_player;
truncate table silver.dim_venue;